In [1]:
############LIBRARIES################
import sys
sys.path.append("..\\..\\CreateNERSilverSet_NewPipelineOutput")
sys.path.append("..\\..\\PrepareBERT_forDatabricks")

from NERSilverSetFromSentences import *
from GrantAndOrganizationMentionsFromGoldSet import grant_and_organization_mentions_from_gold_set
from AnnotateForBert import * 

import pickle
from transformers import BertTokenizerFast
import pandas as pd
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [2]:
############VARIABLES################
#Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
#Define tag ids
id2tag = {0: 'I_GRT', 1: 'O', 2: 'B_GRT', 3: 'B_ORG', 4: 'I_ORG'}
tag2id = {'I_GRT': 0, 'O': 1, 'B_GRT': 2, 'B_ORG': 3, 'I_ORG': 4, -100:-100}

C:\Users\aydxng\Anaconda3\envs\Python37\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\aydxng\Anaconda3\envs\Python37\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [3]:
############FUNCTIONS################

#Add [CLS] and [SEP] tokens, pad until "pad_len" chars.
def add_and_pad(lst,pad_len,cls,sep,pad):
    new_lst = []
    for item in lst:
        new_item = [cls] + item + [sep]
        while len(new_item) != pad_len:
            new_item.append(pad)
        new_lst.append(new_item)
    return new_lst

#Class for funding bodies dataset
class FB_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels,at_mask,seq_lens):
        self.encodings = encodings
        self.labels = labels
        self.at_mask = at_mask
        self.seq_lens = seq_lens

    def __getitem__(self, idx):
        item = dict()
        item['input_ids'] = torch.tensor(self.encodings[idx])
        item['attention_mask'] = torch.tensor(self.at_mask[idx])
        item['labels'] = torch.tensor(self.labels[idx])
        item['seq_len'] =self.seq_lens[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [4]:
######READ THE DATA##########
path = "C:\\Users\\aydxng\\Documents\\ds-fundingbodies-linkingcomponent-masterthesis\\Thesis\\CreateNERSilverSet_NewPipelineOutput\\"
with open(path+'df_sent_divided.pkl','rb') as f:
    df= pickle.load(f)
    
#Discard the samples in "Validation" folder
folders_validation = ['Validation\\Round_2','Validation\\Round_3','Validation\\Round_6','Validation\\Round_7','Validation\\Round_9',
                      'Validation\\Round_11','Validation\\Round_13','Validation\\Round_17','Validation\\Round_18','Validation\\Round_19',
                      'Validation\\Round_20','Validation\\Round_22','Validation\\Round_23','Validation\\Round_25','Validation\\Round_27',
                      'Validation\\Round_28','Validation\\Round_30','Validation\\Round_32']
df = df[~df.Folder.isin(folders_validation)].copy(deep=True)

#Split train/valid/test
train = df[df.Dataset=='Training'].reset_index(drop=True).copy(deep=True)
valid = df[df.Dataset=='Validation'].reset_index(drop=True).copy(deep=True)

#Drop not-needed columns
train.drop(['Dataset','Pipeline_Span_Tags_IOB','Pipeline_Span_Tags_GRT_IOB','Pipeline_Span_Tags_ORG_IOB'],
       axis=1, inplace=True)
valid.drop(['Dataset','Pipeline_Span_Tags_IOB','Pipeline_Span_Tags_GRT_IOB','Pipeline_Span_Tags_ORG_IOB'],
       axis=1, inplace=True)
del df

In [5]:
######GET GOLD NAMED ENTITIES##########
path_to_gold = "C:\\Users\\aydxng\\OneDrive - Reed Elsevier Group ICO Reed Elsevier Inc\\Desktop\\ElsevierData\\Dataset2020July\\Dataset2020July\\"
folders_kpi = ['GoldSet_2019\\KPI\\']
folders_trainin_train = ['GoldSet_2019\\Trainin-Train\\']
folders_trainin_validation = ['GoldSet_2019\\Trainin-Validation\\']
folders_trainin_test = ['GoldSet_2019\\Trainin-Test\\']
folders_regression = ['GoldSet_2019\\Regression\\']
folders_improvement = ['Improvement\\Round_1\\','Improvement\\Round_10\\','Improvement\\Round_12\\','Improvement\\Round_14\\','Improvement\\Round_15\\','Improvement\\Round_21\\','Improvement\\Round_24\\','Improvement\\Round_26\\','Improvement\\Round_29\\','Improvement\\Round_31\\','Improvement\\Round_4\\','Improvement\\Round_5\\','Improvement\\Round_8\\' ]
#Gold named entities per folder
org_dict_kpi, grant_dict_kpi = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_kpi])
org_dict_trainin_train, grant_dict_trainin_train = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_trainin_train])
org_dict_trainin_validation, grant_dict_trainin_validation = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_trainin_validation])
org_dict_trainin_test, grant_dict_trainin_test = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_trainin_test])
org_dict_regression, grant_dict_regression = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_regression])
org_dict_improvement, grant_dict_improvement = grant_and_organization_mentions_from_gold_set([path_to_gold+x for x in folders_improvement])
org_dict_gold = dict()
grant_dict_gold = dict()
org_dict_gold.update(org_dict_kpi)
grant_dict_gold.update(grant_dict_kpi)
org_dict_gold.update(org_dict_trainin_train)
grant_dict_gold.update(grant_dict_trainin_train)
org_dict_gold.update(org_dict_trainin_validation)
grant_dict_gold.update(grant_dict_trainin_validation)
org_dict_gold.update(org_dict_trainin_test)
grant_dict_gold.update(grant_dict_trainin_test)
org_dict_gold.update(org_dict_regression)
grant_dict_gold.update(grant_dict_regression)
org_dict_gold.update(org_dict_improvement)
grant_dict_gold.update(grant_dict_improvement)

Error in file:  00812K_622654249.ann
Error in file:  00901K_618203602.ann
Error in file:  07147K_618282364.ann
Error in file:  09209K_358501993.ann
Error in file:  06041T_618636215.ann
Error in file:  06821T_620696511.ann
Error in file:  07347T_621231937.ann
Error in file:  07451R_605779630.ann
Error in file:  07964R_355725573.ann
Error in file:  09112R_619440406.ann
Error in file:  16263R_359218191.ann
Error in file:  17677R_354095974.ann
Error in file:  17677R_354095974.ann
Error in file:  S0141933115000630.ann
Error in file:  S0141933115000630.ann
Error in file:  S0141933115000630.ann
Error in file:  S0165011412002217.ann
Error in file:  S0166093415003067.ann
Error in file:  S0166218X1400016X.ann
Error in file:  S0168192316303598.ann
Error in file:  S0168192316303598.ann
Error in file:  S0169260714002946.ann
Error in file:  S0267364914001277.ann
Error in file:  S0268005X16300492.ann
Error in file:  S0301679X1500242X.ann
Error in file:  S0304397511003926.ann
Error in file:  S03043975

In [ ]:
########PROCESS TRAINING SET##########
#Add BERT tokenization
res = tokenize_input_bert(train,'Sentence',tokenizer)
train['Sentence_Tokenized'] = res[0]
train['Token_Spans'] = res[1]
#Token encodings does not have [CLS] an [SEP at the moment]
train['Token_Encoding'] = res[2]

#Get Gold span tags for organizations
train['Gold_Span_Tags_ORG_IOB'] = get_span_tags_iob_bert(train,org_dict_gold,"","Token_Spans")
#Get Gold span tags for grants
train['Gold_Span_Tags_GRT_IOB'] = get_span_tags_iob_bert(train,grant_dict_gold,"","Token_Spans")
#Fix the tags
train['Gold_Span_Tags_ORG_IOB'] = fix_iob_tags(train['Gold_Span_Tags_ORG_IOB'].values)
train['Gold_Span_Tags_GRT_IOB'] = fix_iob_tags(train['Gold_Span_Tags_GRT_IOB'].values)
#Merge the tags
train['Gold_Span_Tags_IOB'] = merge_grant_and_org_annotations_iob_bert(train,'Gold_Span_Tags_GRT_IOB','Gold_Span_Tags_ORG_IOB')

#Define max token length (512-2=510)
#Split big sentences
max_len = 510
train, too_long_train =split_long_sentences(train,max_len)

#Separate a part for monitoring the training
train_monitor_idx = np.random.choice(train[train.Folder=='GoldSet_2019\\KPI'].Filename.unique(),1000,replace=False)
train_monitor = train[train.Filename.isin(train_monitor_idx)].reset_index(drop=True).copy(deep=True)
train = train[~train.Filename.isin(train_monitor_idx)].reset_index(drop=True).copy(deep=True)

max_len = 512

#Get tokenized sentences
train_texts = train['Sentence_Tokenized'].values
train_monitor_texts = train_monitor['Sentence_Tokenized'].values

#Get gold tags
train_tags = train['Gold_Span_Tags_IOB'].values
train_monitor_tags = train_monitor['Gold_Span_Tags_IOB'].values

#Get the encoded values for each token
train_encodings = train['Token_Encoding'].values
train_monitor_encodings = train_monitor['Token_Encoding'].values

#Get lengths of sequences
train_seq_lens = train['Token_Encoding'].apply(lambda x: len(x)+2)
train_monitor_seq_lens = train_monitor['Token_Encoding'].apply(lambda x: len(x)+2)

#Pad the encodings and add [CLS] and [SEP] token
train_encodings = add_and_pad(train_encodings,max_len,101,102,0)
train_monitor_encodings = add_and_pad(train_monitor_encodings,max_len,101,102,0)

#Set attention mask to 0 for padding
train_attention_mask = [[0 if num==0 else 1 for num in lst]  for lst in train_encodings]
train_monitor_attention_mask = [[0 if num==0 else 1 for num in lst]  for lst in train_monitor_encodings]

#Get the labels, padded and special tokens added
train_labels = add_and_pad(train_tags,max_len,-100,-100,-100)
train_monitor_labels = add_and_pad(train_monitor_tags,max_len,-100,-100,-100)

#Convert labels to ids
train_labels = [[tag2id[x] for x in label]  for label in train_labels]
train_monitor_labels = [[tag2id[x] for x in label]  for label in train_monitor_labels]

#Create datasets
train_dataset = FB_Dataset(train_encodings, train_labels,train_attention_mask,train_seq_lens)
train_monitor_dataset = FB_Dataset(train_monitor_encodings, train_monitor_labels,train_monitor_attention_mask,train_monitor_seq_lens)

#Write to file
with open("bert_training_data.pkl","wb") as f:
    pickle.dump(train_dataset,f)
    pickle.dump(train_monitor_dataset,f)

In [ ]:
########PROCESS VALIDATION SET##########
#This variable will contain validation set with long sentences
valid_withlong = valid.copy(deep=True)
del valid

#Add BERT tokenization to validation set
res = tokenize_input_bert(valid_withlong,'Sentence',tokenizer)
valid_withlong['Sentence_Tokenized'] = res[0]
valid_withlong['Token_Spans'] = res[1]
#Token encodings does not have [CLS] an [SEP at the moment]
valid_withlong['Token_Encoding'] = res[2]

#Define max token length (512-2=510)
#Split big sentences
#valid -> big sentences splitted
#valid_withlong -> original
max_len = 510
valid, too_long_valid =split_long_sentences_old(valid_withlong,max_len)

#Add tags to both of them
#Get Gold span tags for organizations
valid['Gold_Span_Tags_ORG_IOB'] = get_span_tags_iob_bert(valid,org_dict_gold,"","Token_Spans")
#Get Gold span tags for grants
valid['Gold_Span_Tags_GRT_IOB'] = get_span_tags_iob_bert(valid,grant_dict_gold,"","Token_Spans")
#Fix the tags
valid['Gold_Span_Tags_ORG_IOB'] = fix_iob_tags(valid['Gold_Span_Tags_ORG_IOB'].values)
valid['Gold_Span_Tags_GRT_IOB'] = fix_iob_tags(valid['Gold_Span_Tags_GRT_IOB'].values)
#Merge the tags
valid['Gold_Span_Tags_IOB'] = merge_grant_and_org_annotations_iob_bert(valid,'Gold_Span_Tags_GRT_IOB','Gold_Span_Tags_ORG_IOB')
#Get Gold span tags for organizations
valid_withlong['Gold_Span_Tags_ORG_IOB'] = get_span_tags_iob_bert(valid_withlong,org_dict_gold,"","Token_Spans")
#Get Gold span tags for grants
valid_withlong['Gold_Span_Tags_GRT_IOB'] = get_span_tags_iob_bert(valid_withlong,grant_dict_gold,"","Token_Spans")
#Fix the tags
valid_withlong['Gold_Span_Tags_ORG_IOB'] = fix_iob_tags(valid_withlong['Gold_Span_Tags_ORG_IOB'].values)
valid_withlong['Gold_Span_Tags_GRT_IOB'] = fix_iob_tags(valid_withlong['Gold_Span_Tags_GRT_IOB'].values)
#Merge the tags
valid_withlong['Gold_Span_Tags_IOB'] = merge_grant_and_org_annotations_iob_bert(valid_withlong,'Gold_Span_Tags_GRT_IOB','Gold_Span_Tags_ORG_IOB')

#Prepare "valid" for the model
max_len = 512
valid_texts = valid['Sentence_Tokenized'].values
valid_tags = valid['Gold_Span_Tags_IOB'].values
valid_encodings = valid['Token_Encoding'].values
valid_seq_lens = valid['Token_Encoding'].apply(lambda x: len(x)+2)
valid_encodings = add_and_pad(valid_encodings,max_len,101,102,0)
valid_attention_mask = [[0 if num==0 else 1 for num in lst]  for lst in valid_encodings]
valid_labels = add_and_pad(valid_tags,max_len,-100,-100,-100)
valid_labels = [[tag2id[x] for x in label]  for label in valid_labels]

#Create datasets
val_dataset = FB_Dataset(valid_encodings, valid_labels,valid_attention_mask,valid_seq_lens)

#Write to file
with open("bert_validation_data.pkl","wb") as f:
    pickle.dump(val_dataset,f)
    pickle.dump(valid_withlong,f)
    pickle.dump(valid,f)
    pickle.dump(too_long_valid,f)